# The Goal here is to Learn to Separate Audio in a Efficient Way

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np

import pyAudioAnalysis
import pydub
import ffmpeg

import os

import matplotlib.pyplot as plt
from numpy import sin, cos, pi, linspace, arange, log10, absolute, where
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

# PITCH OR WORD SEGMENTATION ANALYSIS

## This algorithm analyses and segmentate all data with two configurations 

In [3]:
filename = 'audio/training2/J_cata_02.wav'

#List of Words
words = ["chata","chapa","chave","lata","lapa","lava","casa","capa","cave","cata","chuta","chupa","chuva","farta","farpa","farda","ripa","rita","rica"]

#List of Dataset Audio versions
d = {"one": 1, "two": 2, "three": 3, "Four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8,
     "nine": 9, "ten": 10}

iterable = d.keys()

#List of people
p = ['A','J','N']


### (Pitch ID/Word ID) -> Purpose
### (Save Segmentation Pictures / Save Segmentated Audio) -> Process_Select 

In [4]:
#Directorio do Projeto (Escrever o directório do projeto)
os.chdir(r'C:\Users\Dasil\1. Processamento Digital do Sinal\Project')
#os.chdir(r'C:/Users/lemos/PDS/TP/')

Process_Select = False # False for audio save / True for picture save
purpose = False # False for Pitch Segmentation / True for Word Segmentation

for y in words:
    for item in iterable:
        for x in p:
            check = x # For Validation

            if(d[item] <10):
                k = '_0'
            else:
                k = '_'
            
            # Write file path
            l = str(d[item])
            
            if(purpose == False):
                #for Test DataSet
                filename = 'audio/training/'+x+'_'+y+k+l+'.wav' 
            elif(purpose == True):
                filename = 'audio/PD_data/'+x+'_'+y+k+l+'_1.wav' #for Training DataSet


            # Read file path
            fs, audio1 = read(filename)
            f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')

            # Apply butterworth Filter
            wn = 1500/(fs/2)
            b3,a3 = butter(4, wn)
            audio1_filt = lfilter(b3,a3,audio1)
            
            # Calculate Results
            if(purpose == False):
                #Pitch Segmentation
                results = [1 if item > 500 else 0 for item in audio1_filt] #350
                silence_len = 10000 #9000
                min_len = 10000 #2000
            elif(purpose == True):
                #Word Segmentation
                results = [1 if item > 0.001*1e8 else 0 for item in audio1_filt**2]
                silence_len = 2000
                min_len = 5000
            
            try: 
                # Variables
                x1 = x2 = 0
                flag = False
                coordinates = []
                zeros_list = []

                plot(results)
                for i, result in enumerate(results):
                    # First signal rise
                    if(flag == False and result == 1 and x1 == 0):
                        flag = True
                        x1 = i
                    # Check if fall is permanent
                    if(flag == True):
                        if(result == 0):
                            zeros_list.append(result)
                        else:
                            zeros_list = []
                    # Signal fall
                    if(flag == True and len(zeros_list) >= silence_len and x2 == 0):
                        flag = False
                        x2 = i - len(zeros_list)
                        # If data is valid, save it in list
                        if(not x2-x1 <= min_len):
                            coordinates.append([x1, x2])
                            plot([x1, x1], [0, 1], [x2, x2], [0, 1], marker='o')
                        # Reset to initial state
                        x1 = x2 = 0         

                pt1 = coordinates[0]
                try:
                    pt2 = coordinates[1]
                    div_ptn = ((pt2[0] - pt1[1]) / 2) + pt1[1]
                    #print(filename)
                except:
                    div_ptn = ((pt1[0] + pt1[1]) / 2)
                # Draw dividing line
                plot([div_ptn, div_ptn], [0, 1], marker='o', color="black")

                l = d[item]

                if (Process_Select == True):
                    if(purpose == False):
                        #Save Graphic Ilustration
                        plt.savefig('Pitch Detection/crop_pics/'+x+y+l+'.jpeg')
                    elif(purpose == True):
                        #Save Graphic Ilustration
                        plt.savefig('Word Detection/crop_pics/'+x+y+l+'.jpeg')

                elif(Process_Select == False):
                    #Make segmentation point integer
                    div_ptn = round(div_ptn)

                    # Fix-sized segmentation (breaks a signal into non-overlapping segments)
                    signal = audio1 / (2**15)
                    signal_len = len(signal)
                    segment_size_t = 1 # segment size in seconds
                    segment_size = segment_size_t * fs  # segment size in samples

                    # Break signal into list of segments in a single-line Python code
                    segment1 = audio1[:div_ptn]
                    segment2 = audio1[div_ptn:]
                    segments = [segment1, segment2]

                    # Find out the number of Segments
                    n = len(segments)
                    
                    #Convert L to String
                    l = str(l)

                    # Process each chunk with your parameters
                    # Save each segment in a seperate filename
                    if(purpose == False):
                        #Save Files in PD_data_file
                        for iS, s in enumerate(segments):
                            write('Word Detection/PD_data/'+x+'_'+y+k+l+'_{0:d}.wav'.format(segment_size_t * iS, segment_size_t * (iS + 1)), fs, (s))
                    elif(purpose == True):
                        #Save Files in PD_data_file
                        if(d[item] <10):
                            k = '_0'
                        elif(d[item] > 10):
                            k='_'
                        for iS, s in enumerate(segments):
                            write('Word Detection/WD_data/'+x+'_'+y+k+l+'_{0:d}.wav'.format(segment_size_t * iS, segment_size_t * (iS + 1)), fs, (s))
            except:
                print(filename)
        
            #Clean the graphics
            plt.close()
            plt.cla()
            plt.clf()

# of errors: 87


<Figure size 432x288 with 0 Axes>